<a href="https://colab.research.google.com/github/kuuumaaaa/data_analyze_technique_in_kaggle/blob/main/ch01/ch01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

学習データとテストデータの読み込み






In [ ]:
train = pd.read_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/kagglebook-master/input/ch01-titanic/train.csv')
train_x = train.drop(['Survived'],axis =1)
train_y = train['Survived']
test = pd.read_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/kagglebook-master/input/ch01-titanic/test.csv')
test_x = test.copy()

# EDAのポイント
カラムの把握、各カラムの方や値の分布、欠損値、外れ値、目的変数と各変数の相関や関係性
- 変数の平均・標準偏差・最大・最小・分位点
- カテゴリ変数の値の種類数
- 変数の欠損値の値
- 変数間の相関関係

バリデーションデータへの分け方

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

#特徴量の作成
今回GBDTのモデルで作成
- IDは予測に寄与しないため削除
- Name,Ticket,Cabinについて煩雑な処理が必要なため削除
- 文字はlabel encoding
- 欠損はそのまま

In [ ]:
from  sklearn.preprocessing import LabelEncoder

# 変数PassengerIdを除去
train_x = train_x.drop(['PassengerId'],axis=1)
test_x = test_x.drop(['PassengerId'],axis=1)

# 変数Name,Ticket,Cabiを削除
train_x = train_x.drop(['Name','Ticket','Cabin'],axis=1)
test_x = test_x.drop(['Name','Ticket','Cabin'],axis=1)

# それぞれのカテゴリ変数にlabel encoding
for c in ['Sex','Embarked']:
  # 学習データに基づいてどう変換するかを定める
  le = LabelEncoder()
  le.fit(train_x[c].fillna('NA'))

  # 学習データ、テストデータを変換する
  train_x[c] = le.transform(train_x[c].fillna('NA'))
  test_x[c] = le.transform(test_x[c].fillna('NA'))

#モデルの作成
- GBDTのライブラリの一つであるxgboostを用いてモデルを学習

In [ ]:
from xgboost import XGBClassifier

# モデルの作成及び学習データを与えての学習
model = XGBClassifier(n_estimators=20, random_state=71)
# n_estimators : Number of gradient boosted trees. Equivalent to number of boosting rounds.(決定木の数)
# random_state (Optional[Union[numpy.random.RandomState, int]]) –Random number seed.
model.fit(train_x,train_y)

# テストデータの予測値を確率で出力
pred = model.predict_proba(test_x)[:,1]
# Predict the probability of each X example being of a given class.

# テストデータの予測値を二値に変換する
pred_label = np.where(pred > 0.5,1,0)
# 0.5以上を1に、それ以下を0にする

# 提出用ファイルの作成
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':pred_label})
submission.to_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/outputs/ch01/submission_first.csv',index=False)